In [1]:
from PySide6 import QtCore, QtGui, QtWidgets, __version__ 

In [2]:
%matplotlib qt

In [3]:
import mne
import pandas as pd
import pybv

In [4]:
# Specify the path to your BrainVision data
data_path = 'm_01_02/'

# Load the data
raw = mne.io.read_raw_brainvision(data_path + 'm_01_02_pos2a.vhdr', preload=True)

# Print basic information about the data
print(raw.info)

Extracting parameters from m_01_02/m_01_02_pos2a.vhdr...
Setting channel info structure...
Reading 0 ... 1045419  =      0.000 ...  1045.419 secs...


/var/folders/2b/4tnqzv_x3x17pf4q1_n7m8h40000gn/T/ipykernel_68133/424177381.py:5: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(data_path + 'm_01_02_pos2a.vhdr', preload=True)


<Info | 8 non-empty values
 bads: []
 ch_names: Fp1, Fp2, F3, F4, C3, C4, P3, P4, O1, O2, F7, F8, T7, T8, P7, ...
 chs: 63 EEG, 1 misc
 custom_ref_applied: False
 dig: 66 items (3 Cardinal, 63 EEG)
 highpass: 0.0 Hz
 lowpass: 1000.0 Hz
 meas_date: 2023-04-14 11:01:00 UTC
 nchan: 64
 projs: []
 sfreq: 1000.0 Hz
>


/var/folders/2b/4tnqzv_x3x17pf4q1_n7m8h40000gn/T/ipykernel_68133/424177381.py:5: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(data_path + 'm_01_02_pos2a.vhdr', preload=True)


In [5]:
events = mne.events_from_annotations(raw)

Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1', 'Stimulus/S  5', 'Stimulus/S 10', 'Stimulus/S 11', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109', 'Stimulus/S117']


In [6]:
import os

def recursive_search(directory, target_string):
    # Iterate over all files and directories in the given directory
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.csv') and target_string in file:
                file_path = os.path.join(root, file)
                return file_path

    # If the target string is not found in any CSV file
    return None

# Define the target strings
target_strings = ['1a', '1b', '2a', '2b']

# Specify the directory to search
directory_path = 'm_01_02/'

csv_files = {}
# Perform the recursive search for each target string
for target_string in target_strings:
    result = recursive_search(directory_path, target_string)
    if result is not None:
        print(f"Found '{target_string}' in file: {result}")
        csv_files[target_string]=result

Found '2a' in file: m_01_02/01_2a_posner_task_2023-04-14_11h02.53.670/01_2a_posner_task_2023-04-14_11h02.53.670.csv
Found '2b' in file: m_01_02/01_2b_posner_task_2023-04-14_11h59.21.726/01_2b_posner_task_2023-04-14_11h59.21.726.csv


In [21]:
events, event_id = mne.events_from_annotations(raw) # get events
type(events[100][2])

Used Annotations descriptions: ['1', '10', '101', '109', '11', '117', '20', '30', '40', '5', '50', '51', '52', '60', '70', '80', '99999']


numpy.int64

In [7]:
# Read posner data
posner_csv = pd.read_csv('/Users/babe/src/gla/dissertation/init_data_exploration/m_01_02/01_2b_posner_task_2023-04-14_11h59.21.726/01_2b_posner_task_2023-04-14_11h59.21.726.csv')
posner_csv = posner_csv[["block_name", "cue_dir", "stim_pos", "session", "valid_cue"]]
validity = posner_csv[posner_csv["block_name"]=="trials"]["valid_cue"].to_numpy()

# get events 
events, event_id = mne.events_from_annotations(raw, event_dict) # get events
idx = 0 # stim even idx increment

for i, e in enumerate(events):
    code=e[2]
    # If stim
    if(code==51):
        # Change code 51 (end of stim) to 52
        events[i][2]=52
    if(code==50):
        if validity[idx]==True:
            # Give code 51 if valid
            events[i][2]=51
        idx+=1

Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1', 'Stimulus/S  5', 'Stimulus/S 10', 'Stimulus/S 11', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109', 'Stimulus/S117']


In [8]:
events[100]

array([445657,      0,     52])

In [9]:
event_id

{'New Segment/': 99999,
 'Stimulus/S  1': 1,
 'Stimulus/S  5': 5,
 'Stimulus/S 10': 10,
 'Stimulus/S 11': 11,
 'Stimulus/S 20': 20,
 'Stimulus/S 30': 30,
 'Stimulus/S 40': 40,
 'Stimulus/S 50': 50,
 'Stimulus/S 51': 51,
 'Stimulus/S 60': 60,
 'Stimulus/S 70': 70,
 'Stimulus/S 80': 80,
 'Stimulus/S101': 101,
 'Stimulus/S109': 109,
 'Stimulus/S117': 117}

In [10]:
# for i in events:
#     print(i)

event_dict_map = {
    1: '1',
    10: '10',
    101: '101',
    109: '109',
    11: '11',
    117: '117',
    20: '20',
    30: '30',
    40: '40',
    5: '5',
    50: '50',
    51: '51',
    52: '52',
    60: '60',
    70: '70',
    80: '80',
    99999: '99999'
}

In [11]:
s_freq = raw.info['sfreq']
anno_from_events=mne.annotations_from_events(events,sfreq=s_freq,orig_time=raw.info['meas_date'], event_desc=event_dict_map)
raw = raw.set_annotations(anno_from_events, verbose=1)

Cropping annotations 2023-04-14 11:01:00.552357+00:00 - 2023-04-14 11:18:25.972357+00:00
  [0] Keeping  (2023-04-14 11:01:00.552357+00:00 - 2023-04-14 11:01:00.552357+00:00 -> 0.0 - 0.0)
  [1] Keeping  (2023-04-14 11:05:46.085357+00:00 - 2023-04-14 11:05:46.085357+00:00 -> 285.533 - 285.533)
  [2] Keeping  (2023-04-14 11:05:46.086357+00:00 - 2023-04-14 11:05:46.086357+00:00 -> 285.534 - 285.534)
  [3] Keeping  (2023-04-14 11:05:46.093357+00:00 - 2023-04-14 11:05:46.093357+00:00 -> 285.541 - 285.541)
  [4] Keeping  (2023-04-14 11:05:46.094357+00:00 - 2023-04-14 11:05:46.094357+00:00 -> 285.542 - 285.542)
  [5] Keeping  (2023-04-14 11:05:51.243357+00:00 - 2023-04-14 11:05:51.243357+00:00 -> 290.691 - 290.691)
  [6] Keeping  (2023-04-14 11:05:52.718357+00:00 - 2023-04-14 11:05:52.718357+00:00 -> 292.166 - 292.166)
  [7] Keeping  (2023-04-14 11:05:56.138357+00:00 - 2023-04-14 11:05:56.138357+00:00 -> 295.586 - 295.586)
  [8] Keeping  (2023-04-14 11:05:56.238357+00:00 - 2023-04-14 11:05:56.

  [76] Keeping  (2023-04-14 11:07:46.419357+00:00 - 2023-04-14 11:07:46.419357+00:00 -> 405.867 - 405.867)
  [77] Keeping  (2023-04-14 11:07:48.259357+00:00 - 2023-04-14 11:07:48.259357+00:00 -> 407.707 - 407.707)
  [78] Keeping  (2023-04-14 11:07:49.766357+00:00 - 2023-04-14 11:07:49.766357+00:00 -> 409.214 - 409.214)
  [79] Keeping  (2023-04-14 11:07:52.939357+00:00 - 2023-04-14 11:07:52.939357+00:00 -> 412.387 - 412.387)
  [80] Keeping  (2023-04-14 11:07:53.039357+00:00 - 2023-04-14 11:07:53.039357+00:00 -> 412.487 - 412.487)
  [81] Keeping  (2023-04-14 11:07:54.769357+00:00 - 2023-04-14 11:07:54.769357+00:00 -> 414.217 - 414.217)
  [82] Keeping  (2023-04-14 11:07:56.269357+00:00 - 2023-04-14 11:07:56.269357+00:00 -> 415.717 - 415.717)
  [83] Keeping  (2023-04-14 11:07:59.629357+00:00 - 2023-04-14 11:07:59.629357+00:00 -> 419.077 - 419.077)
  [84] Keeping  (2023-04-14 11:07:59.729357+00:00 - 2023-04-14 11:07:59.729357+00:00 -> 419.177 - 419.177)
  [85] Keeping  (2023-04-14 11:08:01.

  [153] Keeping  (2023-04-14 11:10:00.510357+00:00 - 2023-04-14 11:10:00.510357+00:00 -> 539.958 - 539.958)
  [154] Keeping  (2023-04-14 11:10:03.819357+00:00 - 2023-04-14 11:10:03.819357+00:00 -> 543.267 - 543.267)
  [155] Keeping  (2023-04-14 11:10:03.920357+00:00 - 2023-04-14 11:10:03.920357+00:00 -> 543.368 - 543.368)
  [156] Keeping  (2023-04-14 11:10:05.510357+00:00 - 2023-04-14 11:10:05.510357+00:00 -> 544.958 - 544.958)
  [157] Keeping  (2023-04-14 11:10:07.010357+00:00 - 2023-04-14 11:10:07.010357+00:00 -> 546.458 - 546.458)
  [158] Keeping  (2023-04-14 11:10:10.720357+00:00 - 2023-04-14 11:10:10.720357+00:00 -> 550.168 - 550.168)
  [159] Keeping  (2023-04-14 11:10:10.819357+00:00 - 2023-04-14 11:10:10.819357+00:00 -> 550.267 - 550.267)
  [160] Keeping  (2023-04-14 11:10:12.010357+00:00 - 2023-04-14 11:10:12.010357+00:00 -> 551.458 - 551.458)
  [161] Keeping  (2023-04-14 11:10:13.510357+00:00 - 2023-04-14 11:10:13.510357+00:00 -> 552.958 - 552.958)
  [162] Keeping  (2023-04-14

  [229] Keeping  (2023-04-14 11:12:13.860357+00:00 - 2023-04-14 11:12:13.860357+00:00 -> 673.308 - 673.308)
  [230] Keeping  (2023-04-14 11:12:13.960357+00:00 - 2023-04-14 11:12:13.960357+00:00 -> 673.408 - 673.408)
  [231] Keeping  (2023-04-14 11:12:15.940357+00:00 - 2023-04-14 11:12:15.940357+00:00 -> 675.388 - 675.388)
  [232] Keeping  (2023-04-14 11:12:17.441357+00:00 - 2023-04-14 11:12:17.441357+00:00 -> 676.889 - 676.889)
  [233] Keeping  (2023-04-14 11:12:20.840357+00:00 - 2023-04-14 11:12:20.840357+00:00 -> 680.288 - 680.288)
  [234] Keeping  (2023-04-14 11:12:20.940357+00:00 - 2023-04-14 11:12:20.940357+00:00 -> 680.388 - 680.388)
  [235] Keeping  (2023-04-14 11:12:22.441357+00:00 - 2023-04-14 11:12:22.441357+00:00 -> 681.889 - 681.889)
  [236] Keeping  (2023-04-14 11:12:23.941357+00:00 - 2023-04-14 11:12:23.941357+00:00 -> 683.389 - 683.389)
  [237] Keeping  (2023-04-14 11:12:27.440357+00:00 - 2023-04-14 11:12:27.440357+00:00 -> 686.888 - 686.888)
  [238] Keeping  (2023-04-14

  [305] Keeping  (2023-04-14 11:14:17.691357+00:00 - 2023-04-14 11:14:17.691357+00:00 -> 797.139 - 797.139)
  [306] Keeping  (2023-04-14 11:14:17.791357+00:00 - 2023-04-14 11:14:17.791357+00:00 -> 797.239 - 797.239)
  [307] Keeping  (2023-04-14 11:14:19.481357+00:00 - 2023-04-14 11:14:19.481357+00:00 -> 798.929 - 798.929)
  [308] Keeping  (2023-04-14 11:14:20.981357+00:00 - 2023-04-14 11:14:20.981357+00:00 -> 800.429 - 800.429)
  [309] Keeping  (2023-04-14 11:14:24.131357+00:00 - 2023-04-14 11:14:24.131357+00:00 -> 803.579 - 803.579)
  [310] Keeping  (2023-04-14 11:14:24.231357+00:00 - 2023-04-14 11:14:24.231357+00:00 -> 803.679 - 803.679)
  [311] Keeping  (2023-04-14 11:14:25.985357+00:00 - 2023-04-14 11:14:25.985357+00:00 -> 805.433 - 805.433)
  [312] Keeping  (2023-04-14 11:14:31.599357+00:00 - 2023-04-14 11:14:31.599357+00:00 -> 811.047 - 811.047)
  [313] Keeping  (2023-04-14 11:14:34.275357+00:00 - 2023-04-14 11:14:34.275357+00:00 -> 813.723 - 813.723)
  [314] Keeping  (2023-04-14

  [382] Keeping  (2023-04-14 11:16:24.782357+00:00 - 2023-04-14 11:16:24.782357+00:00 -> 924.23 - 924.23)
  [383] Keeping  (2023-04-14 11:16:26.282357+00:00 - 2023-04-14 11:16:26.282357+00:00 -> 925.73 - 925.73)
  [384] Keeping  (2023-04-14 11:16:29.372357+00:00 - 2023-04-14 11:16:29.372357+00:00 -> 928.82 - 928.82)
  [385] Keeping  (2023-04-14 11:16:29.472357+00:00 - 2023-04-14 11:16:29.472357+00:00 -> 928.92 - 928.92)
  [386] Keeping  (2023-04-14 11:16:31.282357+00:00 - 2023-04-14 11:16:31.282357+00:00 -> 930.73 - 930.73)
  [387] Keeping  (2023-04-14 11:16:32.782357+00:00 - 2023-04-14 11:16:32.782357+00:00 -> 932.23 - 932.23)
  [388] Keeping  (2023-04-14 11:16:36.402357+00:00 - 2023-04-14 11:16:36.402357+00:00 -> 935.85 - 935.85)
  [389] Keeping  (2023-04-14 11:16:36.502357+00:00 - 2023-04-14 11:16:36.502357+00:00 -> 935.95 - 935.95)
  [390] Keeping  (2023-04-14 11:16:37.782357+00:00 - 2023-04-14 11:16:37.782357+00:00 -> 937.23 - 937.23)
  [391] Keeping  (2023-04-14 11:16:39.282357+0

In [12]:
type(anno_from_events)

mne.annotations.Annotations

In [13]:
mne.export.export_raw('test123.eeg', raw, overwrite=True, verbose=1)

Overwriting existing file.


/Users/babe/src/gla/dissertation/init_data_exploration/nfb_analysis_env/lib/python3.10/site-packages/pybv/_export.py:61: RuntimeWarning: Encountered data in 'short' format. Converting to float32.
  warn(
/Users/babe/src/gla/dissertation/init_data_exploration/nfb_analysis_env/lib/python3.10/site-packages/pybv/io.py:690: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)


## Open and inspect saved data file

In [14]:
raw2 = mne.io.read_raw_brainvision('test123.vhdr')

Extracting parameters from test123.vhdr...
Setting channel info structure...


In [15]:
edict = {
 'Comment/1': 1,
 'Comment/10': 10,
 'Comment/101': 101,
 'Comment/109': 109,
 'Comment/11': 1,
 'Comment/117': 117,
 'Comment/20': 20,
 'Comment/30': 30,
 'Comment/40': 40,
 'Comment/5': 5,
 'Comment/50': 50,
 'Comment/51': 51,
 'Comment/52': 52,
 'Comment/60': 60,
 'Comment/70': 70,
 'Comment/80': 80,
 'Comment/99999': 999,
 'New Segment/': 99999
}

events2, event_id2 = mne.events_from_annotations(raw2, edict)

Used Annotations descriptions: ['Comment/1', 'Comment/10', 'Comment/101', 'Comment/109', 'Comment/11', 'Comment/117', 'Comment/20', 'Comment/30', 'Comment/40', 'Comment/5', 'Comment/50', 'Comment/51', 'Comment/52', 'Comment/60', 'Comment/70', 'Comment/80', 'Comment/99999', 'New Segment/']


In [16]:
events2[100]

array([445557,      0,     51])

In [17]:
events[100]

array([445657,      0,     52])

In [18]:
event_id2

{'Comment/1': 1,
 'Comment/10': 10,
 'Comment/101': 101,
 'Comment/109': 109,
 'Comment/11': 1,
 'Comment/117': 117,
 'Comment/20': 20,
 'Comment/30': 30,
 'Comment/40': 40,
 'Comment/5': 5,
 'Comment/50': 50,
 'Comment/51': 51,
 'Comment/52': 52,
 'Comment/60': 60,
 'Comment/70': 70,
 'Comment/80': 80,
 'Comment/99999': 999,
 'New Segment/': 99999}